In [8]:
import json
from pathlib import Path

import pandas as pd

In [9]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/musique-mini-nshot-sweep/").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
experiments[0]

246 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/musique',
   'name': 'answerable',
   'split': 'validation'},
  'qa': {'model': 'llama-3-70b-tgi',
   'temperature': 0.1,
   'system_prompt': 'direct/helpful-output-format-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'auto/direct.json',
   'n_shot': 0},
  'run': 0},
 'metrics': {'exact_match': 0.56,
  'f1': 0.7149499759499759,
  'fuzzy_match': 0.6466666666666666,
  '2hops': {'exact_match': 0.69,
   'f1': 0.8419797979797979,
   'fuzzy_match': 0.78},
  '3hops': {'exact_match': 0.56,
   'f1': 0.7190822510822511,
   'fuzzy_match': 0.67},
  '4hops': {'exact_match': 0.43,
   'f1': 0.5837878787878787,
   'fuzzy_match': 0.49}}}

In [10]:
df = pd.json_normalize(experiments).drop(columns=["commit"])
mask = (
    (df["params.dataset.path"] == "bdsaglam/musique-mini") 
    & (df["params.qa.few_shot_examples"].str.contains("auto/"))
)

df = df.loc[mask].copy()
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)

param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]
df.dropna(subset=param_cols + metric_cols, inplace=True, how="any")
df.drop_duplicates(subset=param_cols, inplace=True)

print(f"{len(df)} experiments after preprocessing")

142 experiments after preprocessing


In [11]:
def parse_qa_technique(system_prompt_filename):
    return system_prompt_filename.split("/")[0]

df["params.qa.technique"] = df["params.qa.system_prompt"].map(parse_qa_technique)

In [12]:
def sorted_tuple(x):
    return tuple(sorted(x))


run_counts = (
    df.groupby(
        [
            "params.qa.system_prompt",
            "params.qa.user_prompt_template",
            "params.qa.few_shot_examples",
            "params.qa.n_shot",
            "params.qa.temperature",
        ]
    )["params.run"]
    .aggregate(sorted_tuple)
    .reset_index()
)
run_counts.loc[run_counts["params.run"].map(len) != 3]

,params.qa.system_prompt,params.qa.user_prompt_template,params.qa.few_shot_examples,params.qa.n_shot,params.qa.temperature,params.run
15,cot/format-thought.txt,cq.txt,auto/cot.json,1,0.7,"(1, 3)"
19,cot/format-thought.txt,cq.txt,auto/cot.json,3,0.7,"(2, 3)"


In [13]:
for col in run_counts.columns:
    print(f"- {col}")
    for value in run_counts[col].unique():
        print(f"\t{value}")

- params.qa.system_prompt
	ccot/format-thought.txt
	cot/format-thought.txt
	cte/excellent-format-few.txt
	direct/helpful-output-format-few.txt
- params.qa.user_prompt_template
	cq.txt
- params.qa.few_shot_examples
	auto/ccot.json
	auto/cot.json
	auto/cte.json
	auto/direct.json
- params.qa.n_shot
	0
	1
	2
	3
	6
	9
- params.qa.temperature
	0.1
	0.7
- params.run
	(1, 2, 3)
	(1, 3)
	(2, 3)


In [14]:
df.to_json('results.jsonl', orient='records', lines=True)